In [1]:
import random

In [2]:
 R.<X> = Integers(0)[]

In [3]:
def convolution(f,g,N):
    #This could also be done by list comprehension
    return f*g %(X^N -1)

In [4]:
def generate_key(dpos,dneg,N):
    # This is probably not the best way to generate a random key
    # We first randomly place dpos 1s and then dneg -1s,
    # the rest are 0 
    poly = [0]*N
    while dpos > 0:
        i = randrange(N)
        if poly[i] == 0:
            poly[i] = 1
            dpos -= 1
    while dneg >0:
        i = randrange(N)
        if poly[i] == 0:
            poly[i] = -1
            dneg -= 1
    return R(poly)

In [5]:
def poly_mod(f,q):
    # This could also be done by list comprehension
    Rq.<X> = Integers(q)[]
    coefflist = f.list()
    return Rq(coefflist)

In [6]:
def center_lift(f,p):
    coefflist = R(f).list()
    return R([((coefflist[i] + p//2) %p) - p//2 for i in range(len(coefflist))])

In [7]:
def inverse_prime(f,j,N):
    # We could also do this using the extended euclidian algorithm 
    # since // is implemented modulo primes
    Rj.<X> = Integers(j)[]
    RR = Rj.quotient(X^(N) -1)
    return R(lift(RR(f.list())^(-1)))

In [8]:
def inverse_prime_pow(f,t,q,N):
    # The idea here is that we first find an inverse g mod t. 
    # Then we know that g(x)*f(x) = 1 - tr(x). 
    # This means that (1 + tr(x))g(x)*f(x) = 1 - t^2r^2(x)
    # We iterate this process until we get g'(x)f(x) = 1 - qr^m(x), 
    # and g'(x) is the inverse of f(x) mod q
    
    assert q.is_power_of(t)
    g = inverse_prime(f,t,N)
    assert poly_mod(convolution(f,g,N),t) == 1
    while True:
        r = center_lift(convolution(f,g,N),q)
        if r == 1:
            return g
        g = center_lift(convolution(g,2-r,N),q)

In [9]:
def inverse(f,q,N):
    # To find the inverse of f modulo q where q is composite
    # We first find the inverse modulo the prime powers composing q
    # Then we use the CRT to find the inverse mod the product of those prime powers, aka q
    
    #get prime factorization of q
    fact = factor(q)
    factors = list(fact)
    #get the inverses mod prime powers
    inverse_dict = {}
    for l in factors:
        inverse_dict[l[0]] = inverse_prime_pow(f,l[0],(l[0]^l[1]),N)
    #use the CRT to find the total inverse
    inverse = 0
    for i in factors:
        Ni = q // (i[0]^i[1])
        inverse += inverse_dict[i[0]]*Ni*xgcd(Ni,i[0]^i[1])[1]
    return inverse

In [10]:
def NTRU_setup(N, p, q, d):

    # check that we don't have to create impossible polynomials
    assert 2*d + 1 <= N

    # We could check that the decryption will always happen accurately.
    # But it turns out that in practice the decryption 
    # is accurate almost all the time even if q < (6*d+1)*p.
    # This is because it's highly unlikely that all the coefficients will line up to attain that bound 
    # assert q > (6*d+1)*p
    
    #generate key f invertible mod p and mod q of degree N-1 with coefficients 1, 0, -1 
    while True:
        try:
            f = generate_key(d+1,d,N)
            Fp = inverse(f,p,N)
            Fq = inverse(f,q,N)
            break
        except: pass
    
    assert poly_mod(convolution(f,Fp,N),p) == 1
    assert poly_mod(convolution(f,Fq,N),q) == 1

    #generate key g with coefficients 1,0,-1
    g = generate_key(d,d,N)
    
    #create the public key
    h = poly_mod(convolution(Fq,g,N),q)

    return f, Fp, h

In [11]:
def NTRU_encrypt(m,p,q,N,h):
    #generate a random perturbation
    r = generate_key(d,d,N)
    #return the encrypted message
    return poly_mod(p*convolution(h,r,N) + m,q)

In [12]:
def NTRU_decrypt(f,Fp,e,p,q,N):
    a = R(center_lift(convolution(f,e,N),q))
    return center_lift(convolution(a,Fp,N),p)

In [14]:
#change parameters to play with it as you see fit :)
# the current parameters are considered "safe"
p = 3
q = 4096
R.<X> = Integers(0)[]
N = 821
d = 271
f_Fp_h = NTRU_setup(N,p,q,d)
f = f_Fp_h[0]
Fp = f_Fp_h[1]
h = f_Fp_h[2]
#print(h)
m = generate_key(d,d,N)
print(m)
e = NTRU_encrypt(m,p,q,N,h)
#print(e)
NTRU_decrypt(f,Fp,e,p,q,N)

-X^820 - X^818 - X^816 - X^815 + X^814 - X^813 - X^812 - X^811 + X^809 + X^806 - X^803 - X^802 - X^800 - X^799 + X^797 + X^796 + X^793 + X^790 - X^789 + X^788 - X^787 + X^786 - X^785 - X^782 + X^781 - X^780 + X^778 - X^777 + X^776 - X^775 + X^774 - X^772 - X^771 - X^770 + X^768 - X^765 - X^764 + X^763 + X^762 - X^761 + X^760 - X^758 + X^757 - X^756 - X^755 - X^754 - X^753 - X^751 - X^750 + X^749 - X^746 + X^745 - X^744 + X^740 + X^739 - X^737 + X^734 + X^733 - X^732 - X^731 - X^730 - X^728 + X^727 + X^726 - X^722 + X^721 + X^720 - X^717 - X^716 + X^715 + X^714 + X^713 + X^712 + X^711 + X^710 + X^708 + X^707 + X^706 - X^705 - X^704 - X^702 - X^700 - X^698 - X^697 + X^696 + X^694 - X^693 - X^692 + X^691 - X^690 - X^689 + X^688 - X^687 - X^686 - X^684 - X^682 + X^680 + X^679 + X^677 - X^676 + X^675 + X^674 + X^673 + X^668 + X^666 - X^663 - X^661 + X^660 + X^659 - X^657 + X^656 + X^655 + X^653 - X^652 - X^650 + X^649 + X^648 - X^647 - X^646 + X^642 + X^641 - X^639 + X^638 + X^636 + X^634 +

-X^820 - X^818 - X^816 - X^815 + X^814 - X^813 - X^812 - X^811 + X^809 + X^806 - X^803 - X^802 - X^800 - X^799 + X^797 + X^796 + X^793 + X^790 - X^789 + X^788 - X^787 + X^786 - X^785 - X^782 + X^781 - X^780 + X^778 - X^777 + X^776 - X^775 + X^774 - X^772 - X^771 - X^770 + X^768 - X^765 - X^764 + X^763 + X^762 - X^761 + X^760 - X^758 + X^757 - X^756 - X^755 - X^754 - X^753 - X^751 - X^750 + X^749 - X^746 + X^745 - X^744 + X^740 + X^739 - X^737 + X^734 + X^733 - X^732 - X^731 - X^730 - X^728 + X^727 + X^726 - X^722 + X^721 + X^720 - X^717 - X^716 + X^715 + X^714 + X^713 + X^712 + X^711 + X^710 + X^708 + X^707 + X^706 - X^705 - X^704 - X^702 - X^700 - X^698 - X^697 + X^696 + X^694 - X^693 - X^692 + X^691 - X^690 - X^689 + X^688 - X^687 - X^686 - X^684 - X^682 + X^680 + X^679 + X^677 - X^676 + X^675 + X^674 + X^673 + X^668 + X^666 - X^663 - X^661 + X^660 + X^659 - X^657 + X^656 + X^655 + X^653 - X^652 - X^650 + X^649 + X^648 - X^647 - X^646 + X^642 + X^641 - X^639 + X^638 + X^636 + X^634 +